In [ ]:
from math import pi
import numpy as np
from pde import plot_magnitudes,PDE,movie_scalar,UnitGrid,plot_magnitudes, CartesianGrid, MemoryStorage, ScalarField, plot_kymograph, plot_kymographs, FieldCollection, PlotTracker, PDEBase, ExplicitSolver, FileStorage,VectorField
import pandas as pd
import os,sys
import matplotlib.animation as animation
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
from pde.tools.numba import jit
import itertools
import numba as nb
import pickle as pickle

In [ ]:
datafolder = 'data/'
datafile = 'test'
filename = datafolder + datafile
with open(filename, 'rb') as output:
    dic = pickle.load(output)
    phi = dic['phi']
    dphi = dic['dphi']
    alpha = dic['alpha']
    Lx = dic['Lx']
    Ly = dic['Lx']
    Nx= dic['Nx']
    Ny= dic['Ny']
    print(Nx,Ny)
    frames=len(phi)
rephi = np.real(phi)
imphi = np.imag(phi)

redphi = np.real(dphi)
imdphi = np.imag(dphi)

argphi = np.angle(phi)
modphi = np.abs(phi)

argdphi = np.angle(dphi)
moddphi = np.abs(dphi)

Po =  np.real(dphi)*np.imag(phi) - np.imag(dphi) *np.real(phi) 
Pd = np.abs(dphi)**2
sumPo = np.sum(Po,axis=(1,2))
sumPd = np.sum(Pd,axis=(1,2))


normPo = Po/sumPo[:,None,None]
normPd = Pd/sumPd[:,None,None]
dP = Po-Pd
sumdP = sumPo - sumPd
normdP = dP/sumdP[:,None,None]
density = modphi**2


times = np.arange(frames)
momentum = np.abs(np.sum(phi,axis=(1,2)))**2
energy = np.sum(density,axis=(1,2))
error = momentum/energy




In [ ]:
plt.rcParams.update({'font.size': 16})
fig,ax = plt.subplots(4,5,figsize=(21,16),dpi=150)

imqties = [rephi,modphi,argphi,density,energy,
           Po,normPo,sumPo,np.sum(modphi,axis=(1,2)),np.log10(energy),
           Pd,normPd,sumPd,error,
           dP,normdP,sumdP]


titles =['Re($\\phi$)','$|\\phi|$','Arg($\\phi$)','$|\\phi|^2$','$\\int|\\phi|$',
           'odd power: Im($\\phi$)Re($\\dot{\\phi}$)-Im($\\dot{\\phi}$)Re($\\phi$)','$\\frac{P_{\\mathrm{odd}}}{\\int P_{\\mathrm{odd}}}$','$\\int P_{\mathrm{odd}}$','$\\int(|\\phi|)$','$\\mathrm{log}(|\\phi|^2)$',
           'dissipated power: $|\\dot{\\phi}|^2$','$\\frac{P_{\\mathrm{disp}}}{\\int P_{\\mathrm{disp}}}$','$\\int P_{\mathrm{disp}}$','$|\\int \\phi|^2/\\int |\\phi|^2$',
            '$P_{\\mathrm{odd}}-P_{\\mathrm{disp}}$','$\\frac{P_{\\mathrm{odd}}-P_{\\mathrm{disp}}}{\\int P_{\\mathrm{odd}}-P_{\\mathrm{disp}}}$','$\\int P_{\\mathrm{odd}}-P_{\\mathrm{disp}}$']
   
fig.suptitle('2D phi model')
im = [0,1,2,3,5,6,10,11,15,16]
sc = [4,7,8,9,12,13,17]
print(len(titles))
o=0
ims=[]
for n,i in enumerate(ax.flatten()):
    if n==2:
        cmap = 'hsv'
    else:
        cmap='viridis'
    if n in im:
        ims.append(i.imshow(imqties[o][0],interpolation='none',cmap=cmap))
        plt.colorbar(ims[o],ax=i)
        i.set_title(titles[o])
        o+=1
    elif n in sc:
        ims.append(i.scatter(times[0],imqties[o][0],s=5))
        i.set_title(titles[o])
        o+=1
plt.tight_layout()

def animate(i):
    o=0
    for n,j in enumerate(ax.flatten()):
        if n in im:
            vmin = np.min(imqties[o][i])
            vmax = np.max(imqties[o][i])
            ims[o].set_array(imqties[o][i])
            ims[o].set_clim(vmin, vmax)
            o+=1
        elif n in sc:
            ims[o].set_offsets(np.c_[times[:i],imqties[o][:i]])
            if i>1:
                j.set_xlim([np.min(times[:i]),np.max(times[:i])])
                j.set_ylim([np.min(imqties[o][:i]),np.max(imqties[o][:i])])
            o+=1
anim = animation.FuncAnimation(fig, animate, 
                               frames=frames, interval=20, blit=False)
anim.save('output/test.mp4')